In [6]:
#load libraries and data
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

dat = pd.read_csv("FinalDataframe_3.csv")
dat

,Unnamed: 0,Unnamed: 0.1,fips_final,NAME,num_hospitals,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,...,mob_grocery,mob_parks,mob_transit,mob_workplace,mob_residential,0-17,18-44,45-64,65-74,75+
0,0,0,1001,"Autauga County, Alabama",1,85,55,6,6,0,...,5.174545,-34.000000,NaN,-24.734545,10.947619,14613,19347,14065,4013,2533
1,1,1,1003,"Baldwin County, Alabama",4,386,362,51,51,0,...,-5.712121,-3.723232,-14.602020,-23.003030,8.606817,41898,58343,51456,17803,12765
2,2,2,1005,"Barbour County, Alabama",1,74,30,5,5,0,...,-5.681072,NaN,NaN,-17.282701,NaN,6015,9899,7634,2238,1671
3,3,3,1007,"Bibb County, Alabama",1,35,25,4,4,0,...,-4.359533,3.346264,-6.092929,-22.455556,9.632323,5201,8687,6121,1723,1183
4,4,4,1009,"Blount County, Alabama",1,25,25,6,6,0,...,1.341146,28.599359,-48.302083,-24.653646,9.384231,14106,19416,15361,5079,3360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,2296,2296,56027,"Niobrara County, Wyoming",1,24,24,4,4,0,...,NaN,NaN,NaN,NaN,NaN,470,735,766,292,221
2297,2297,2297,56029,"Park County, Wyoming",2,50,50,8,8,0,...,-2.845719,-6.057386,NaN,-24.497670,-0.095238,5911,8608,8744,2776,2166
2298,2298,2298,56033,"Sheridan County, Wyoming",1,88,71,11,11,0,...,18.000000,NaN,NaN,-15.652778,6.214286,6485,9056,9027,2479,2069
2299,2299,2299,56037,"Sweetwater County, Wyoming",1,99,58,10,10,0,...,5.679631,NaN,6.521652,-20.283480,8.058333,11869,16662,11632,2146,1497


In [39]:
#drop NA...
dat = dat.dropna()

#for X, or what we're clustering, we are removing outcomes y, such as case and death totals
#Also removing strings; can't k-means cluster on these
#Also removing columns Unnamed, they seem to be artifacts from data cleaning/combining multiple datasets?
X = dat.drop(['Unnamed: 0', 'Unnamed: 0.1','NAME',  'cases_total',
       'deaths_total', 'cases_per_capita', 'deaths_per_capita',
       'case_fatality_rate', 'Unnamed: 0.1.1', 'county', 'STATEFP', 'COUNTYFP',
       'CountyName', 'StateName', 'State',], axis=1)
y = dat[["cases_total", "deaths_total", "cases_per_capita", "deaths_per_capita", "case_fatality_rate"]]

#set number of clusters
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)

#Run the clustering algorithm
model = kmeans.fit(X)
model

#Generate cluster predictions and store in y_hat
y_hat = kmeans.predict(X)

In [40]:
#function that creates a dataframe with a column for cluster number
cluster_centers = kmeans.cluster_centers_

def kmean_col(X, cluster_centers):
        cols = list(X)
        cols.append('cluster')
        
# Zip with a column called 'cluster' (index)
        Z = [np.append(A, index) for index, A in enumerate(cluster_centers)]
    
# Convert to pandas data frame for plotting
        P = pd.DataFrame(Z, columns=cols)
        P['cluster'] = P['cluster'].astype(int)
        return P
    
P = kmean_col(X, cluster_centers)
P

,fips_final,num_hospitals,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,potential_increase_in_bed_capac,bed_utilization,avg_ventilator_usage,...,mob_parks,mob_transit,mob_workplace,mob_residential,0-17,18-44,45-64,65-74,75+,cluster
0,30267.224072,1.755442,203.110115,165.829706,18.487836,18.487836,3.297055,37.280410,0.420425,2.976780,...,5.105872,-14.287953,-21.470535,8.768410,1.622848e+04,2.414371e+04,1.881714e+04,5421.396927,4409.827145,0
1,27152.666667,23.600000,5292.933333,4798.466667,549.000000,549.000000,241.800000,494.466667,0.616392,6.175509,...,-3.485442,-31.320274,-26.753065,11.959760,4.673326e+05,7.297283e+05,4.805219e+05,119106.933333,102705.133333,1
2,28244.841270,10.476190,2773.571429,2352.476190,269.095238,269.095238,121.777778,421.095238,0.630867,6.789183,...,10.318287,-26.349692,-27.423623,11.680093,2.014910e+05,3.163226e+05,2.209183e+05,53945.920635,49173.507937,2
3,30612.393939,5.151515,1043.648485,906.363636,94.533333,94.533333,36.290909,137.284848,0.611093,5.602769,...,15.079238,-20.198507,-24.829621,10.344226,8.257373e+04,1.226797e+05,9.204139e+04,24253.854545,21085.903030,3
4,23081.666667,57.666667,14244.000000,11856.333333,1332.000000,1332.000000,540.333333,2387.666667,0.582885,6.140437,...,-7.960557,-28.298924,-28.048364,11.969624,1.129325e+06,1.721146e+06,1.045460e+06,258470.333333,213682.000000,4


In [43]:
X['label'] = kmeans.labels_
X

,fips_final,num_hospitals,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,potential_increase_in_bed_capac,bed_utilization,avg_ventilator_usage,...,mob_parks,mob_transit,mob_workplace,mob_residential,0-17,18-44,45-64,65-74,75+,label
1,1003,4,386,362,51,51,0,24,0.562203,2.0,...,-3.723232,-14.602020,-23.003030,8.606817,41898,58343,51456,17803,12765,0
3,1007,1,35,25,4,4,0,10,0.430904,1.0,...,3.346264,-6.092929,-22.455556,9.632323,5201,8687,6121,1723,1183,0
4,1009,1,25,25,6,6,0,0,0.499069,2.0,...,28.599359,-48.302083,-24.653646,9.384231,14106,19416,15361,5079,3360,0
6,1013,1,72,44,7,7,0,28,0.187111,2.0,...,39.938053,-12.338889,-19.988235,8.991837,5056,6597,5805,1829,1660,0
7,1015,3,486,435,30,30,0,51,0.431563,5.0,...,4.506024,-13.395833,-18.810748,8.423611,27126,42361,32095,9437,7553,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,56013,1,146,133,12,12,0,13,0.128829,2.0,...,-11.000000,0.541667,-16.956693,6.673469,10212,12857,11249,3253,2552,0
2292,56019,1,25,22,4,4,0,3,0.275342,1.0,...,38.488789,-7.892308,-19.278571,9.389685,1896,2406,2679,884,704,0
2293,56021,1,206,170,15,15,0,36,0.513360,4.0,...,19.239775,-6.368484,-22.695670,9.661588,22401,32662,25170,6441,5064,0
2294,56023,2,38,35,8,8,0,3,0.212492,2.0,...,-9.852174,-1.841379,-18.962428,5.925373,5104,5646,5115,1372,869,0


1027